In [1]:
import pandas as pd

In [2]:
excel_sheet = pd.read_excel("Datos TallerMCII 2023-1 (1).xlsx", sheet_name = "DATOS PUNTO I")

2. Considere el número mensual de pernoctaciones desde el extranjero a Europa (en millones).
 - a) Utilice la técnica de aditivo estacional y multiplicativo estacional para pronosticar el número de pernoctaciones que se esperan en los próximos 3 meses, con su respectivo margen de error. Determine si los residuos de cada modelo son ruido blanco y seleccione el mejor. Solamente realice los pronósticos con el mejor modelo. Considere 8000 simulaciones y 600 repeticiones. Interprete.
 - b) Utilice la técnica de regresión múltiple con variables ficticias para pronosticar el número de pernoctaciones que se esperan en los próximos 3 meses. Determine si los residuos del modelo son ruido blanco.

In [3]:
pernotaciones_frame = excel_sheet.iloc[:,6:8]
pernotaciones_frame["Periodo"] = pd.to_datetime(pernotaciones_frame["Periodo"], format = "%Y-%m" )

In [4]:
pernotaciones_frame.shape

(569, 2)

In [5]:
pernotaciones_frame = pernotaciones_frame.dropna()

In [6]:
pernotaciones_frame.shape

(200, 2)

In [7]:
pernotaciones_frame.info()

<class 'pandas.core.frame.DataFrame'>
Index: 200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column                                                         Non-Null Count  Dtype         
---  ------                                                         --------------  -----         
 0   Periodo                                                        200 non-null    datetime64[ns]
 1   Número mensual de pernoctaciones desde el extranjero a Europa  200 non-null    float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 4.7 KB


In [8]:
pernotaciones_frame.columns = ["Periodo", "Pernotaciones"]
pernotaciones_frame.set_index("Periodo", inplace = True)
pernotaciones_frame.describe()

,Pernotaciones
count,200.000000
mean,1688.982589
std,425.914580
min,811.080000
25%,1406.153750
50%,1611.952500
75%,1913.162250
max,2601.068000


In [9]:
pernotaciones_frame.resample("M").first()

,Pernotaciones
Periodo,
2006-06-30,1462.712000
2006-07-31,1618.582000
2006-08-31,1867.108000
2006-09-30,1163.087000
2006-10-31,1247.724000
...,...
2022-09-30,1124.965249
2022-10-31,1654.963000
2022-11-30,2601.068000


In [16]:

import matplotlib.pyplot as plt
from statsmodels.tsa.holtwinters import ExponentialSmoothing



In [26]:
# Create the Holt-Winters model with alpha=0 (no trend smoothing)
model = ExponentialSmoothing(pernotaciones_frame['Pernotaciones'], seasonal_periods=12)

# Fit the model to the training data
fitted_model = model.fit()

# Forecast the next 3 months
forecast = fitted_model.forecast(3)

# Print the forecasted values
print("Forecasted values:", forecast) 

Forecasted values: 2023-02-01    2123.707736
2023-03-01    2123.707736
2023-04-01    2123.707736
Freq: MS, dtype: float64


/workspaces/Metodos-cuantitativos/env/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
  self._init_dates(dates, freq)


In [27]:
fitted_model.summary()

Dep. Variable:,Pernotaciones,No. Observations:,200
Model:,ExponentialSmoothing,SSE,28964204.785
Optimized:,True,AIC,2380.651
Trend:,None,BIC,2387.247
Seasonal:,None,AICC,2380.856
Seasonal Periods:,None,Date:,"Sat, 13 May 2023"
Box-Cox:,False,Time:,16:17:36
Box-Cox Coeff.:,None,,
,coeff,code,optimized
smoothing_level,0.9950000,alpha,True
initial_level,1515.8663,l.0,True


In [ ]:
import pandas as pd
from itertools import product
from statsmodels.tsa.holtwinters import ExponentialSmoothing

# Load the monthly data into a DataFrame
df = pd.read_csv('monthly_data.csv', parse_dates=['date'])
df.set_index('date', inplace=True)

# Split the data into a training set and a validation set
train_data = df.iloc[:-12]  # Use all but the last 12 months for training
valid_data = df.iloc[-12:]  # Use the last 12 months for validation

# Define the range of values for alpha, beta, and gamma
alpha_values = [0.1, 0.2, 0.3]
beta_values = [0.1, 0.2, 0.3]
gamma_values = [0.1, 0.2, 0.3]

# Perform grid search to find the optimal parameters
best_params = None
best_error = float('inf')

for alpha, beta, gamma in product(alpha_values, beta_values, gamma_values):
    model = ExponentialSmoothing(train_data['value'], trend='add', seasonal='add', seasonal_periods=12)
    fitted_model = model.fit(smoothing_level=alpha, smoothing_slope=beta, smoothing_seasonal=gamma)
    forecast = fitted_model.forecast(len(valid_data))
    error = mean_squared_error(valid_data['value'], forecast)
    
    if error < best_error:
        best_params = (alpha, beta, gamma)
        best_error = error

# Fit the model with the best parameters using the entire training set
final_model = ExponentialSmoothing(df['value'], trend='add', seasonal='add', seasonal_periods=12)
final_fitted_model = final_model.fit(smoothing_level=best_params[0], smoothing_slope=best_params[1], smoothing_seasonal=best_params[2])

# Forecast future values using the final model
forecast = final_fitted_model.forecast(3)

print("Best parameters:", best_params)
print("Forecasted values:", forecast)
